# Naive approach 1 - Linear Regression

In [2]:
# Get dataset
import torchvision.datasets as datasets
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
import numpy as np

# Define classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Flatten images and separate labels
train_images = np.array([np.array(image).flatten() for image, label in trainset])
train_labels = np.array([label for image, label in trainset])
test_images = np.array([np.array(image).flatten() for image, label in testset])
test_labels = np.array([label for image, label in testset])

print(f"train_images shape: {train_images.shape}")
print(f"train_labels shape: {train_labels.shape}")
print(f"test_images shape: {test_images.shape}")
print(f"test_labels shape: {test_labels.shape}")

train_images shape: (50000, 3072)
train_labels shape: (50000,)
test_images shape: (10000, 3072)
test_labels shape: (10000,)


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl

class LinearRegression(pl.LightningModule):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        out = self.linear(x)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.CrossEntropyLoss()(y_hat, y)
        preds = torch.argmax(y_hat, dim=1)
        acc = torch.sum(preds == y).item() / (len(y) * 1.0)
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss

    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=0.01)
        return optimizer

# Define input size and number of classes
input_size = train_images.shape[1]
num_classes = len(classes)

# Create PyTorch Lightning trainer
trainer = pl.Trainer(max_epochs=100)

# Create linear regression model
model = LinearRegression(input_size, num_classes)

# Train the model
trainset_tensor = torch.utils.data.TensorDataset(torch.Tensor(train_images), torch.Tensor(train_labels).long())
trainloader = torch.utils.data.DataLoader(trainset_tensor, batch_size=32, shuffle=True)
trainer.fit(model, trainloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/tomas/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(

  | Name   | Type   | Params
----------------------------------
0 | linear | Linear | 30.7 K
----------------------------------
30.7 K    Trainable params
0         Non-trainable params
30.7 K    Total params
0.123     Total estimated model params size (MB)
/home/tomas/.local/lib/pyth

Epoch 99: 100%|██████████| 1563/1563 [00:15<00:00, 99.73it/s, v_num=4] 

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 1563/1563 [00:15<00:00, 99.71it/s, v_num=4]


In [6]:
# Convert test images and labels to PyTorch tensors
test_images_tensor = torch.Tensor(test_images)
test_labels_tensor = torch.Tensor(test_labels).long()

# Get model predictions
with torch.no_grad():
    outputs = model(test_images_tensor)
    _, predicted = torch.max(outputs.data, 1)

# Compute accuracy
total = test_labels_tensor.size(0)
correct = (predicted == test_labels_tensor).sum().item()
accuracy = 100 * correct / total

print('Accuracy: {:.2f}%'.format(accuracy))

Accuracy: 38.66%
